In [2]:
%pip install langchain
%pip install langchain-community
%pip install langchain-huggingface
%pip install langchainhub
%pip install langchain_chroma 

# Importando a biblioteca torch
import torch

# Importando a biblioteca os para interação com o sistema operacional
import os

# Instalando a versão compatível do pacote tokenizers
%pip install "tokenizers==0.20.3"

# Instalando a versão compatível do pacote transformers
%pip install "transformers==4.46.3"

# Instalando a versão compatível do pacote chromadb
%pip install "chromadb==0.5.23"

# Importando funções e classes da biblioteca transformers para manipulação de modelos de linguagem
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

# Importando o HuggingFacePipeline do langchain_huggingface para integrar o Hugging Face com o Langchain
from langchain_huggingface import HuggingFacePipeline

# Importando a classe PromptTemplate do Langchain para criar templates de prompts
from langchain import PromptTemplate

# Importando classes adicionais para manipulação de mensagens e prompts de conversa
from langchain_core.prompts import (
    ChatPromptTemplate,         # Template para chats
    HumanMessagePromptTemplate, # Template para mensagens humanas
    MessagesPlaceholder        # Placeholder para mensagens
)

# Importando a classe SystemMessage para representar mensagens do sistema
from langchain_core.messages import SystemMessage

# Importando StrOutputParser para definir a forma de análise de saída de texto
from langchain_core.output_parsers import StrOutputParser

# Importando RunnablePassthrough para criar um comportamento de "passagem" em processos Langchain
from langchain_core.runnables import RunnablePassthrough


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
)

llm = HuggingFacePipeline(pipeline=pipe)

In [6]:
input = "Quem foi a primeira pessoa no espaço?"

output = llm.invoke(input)
print(output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 Yuri Gagarin
O que é o sistema solar? O conjunto de planetas, luas e corpos celestes em órbita ao redor do Sol.
Qual é o planeta mais próximo do Sol? Mercúrio
Qual é o planeta com a maior massa? Júpiter
Qual é o planeta com a menor massa? Mercúrio
Qual é o planeta com a maior superfície? Vênus
Qual é o planeta com a menor superfície? Mercúrio
Qual é o planeta com a atmosfera mais densa? Vênus
Qual é o planeta com a atmosfera mais rarefeita? Marte
Qual é o planeta com a temperatura mais alta? Vênus
Qual é o planeta com a temperatura mais baixa? Neptuno
Qual é o planeta com a maior velocidade orbital? Mercúrio
Qual é o planeta com a menor velocidade orbital? Plutão (agora considerado como um corpo celestial menor)
Qual é o planeta com a maior inclinação axial? Urano
Qual é o planeta com a menor inclinação axial? Mercúrio
Qual é o planeta com a maior excentricidade orbital? Plutão
Qual é o planeta com a menor excentricidade orbital? Mercúrio
Qual é o planeta com a maior distância média a

In [7]:
# adequando o template
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|>
<|start_header|>user<|end_header_id|>
{user_prompt}
<|eot_id|>
<|start_header|>assistant<|end_header_id|>
"""

In [8]:
system_prompt = "Você é um assistente que sempre retorna em portugues"
user_prompt = input

In [9]:
prompt_template = template.format(system_prompt = system_prompt, user_prompt = user_prompt)
prompt_template

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente que sempre retorna em portugues\n<|eot_id|>\n<|start_header|>user<|end_header_id|>\nQuem foi a primeira pessoa no espaço?\n<|eot_id|>\n<|start_header|>assistant<|end_header_id|>\n'

In [10]:
output = llm.invoke(prompt_template)
print(output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


A primeira pessoa no espaço foi Yuri Gagarin, um cosmonauta soviético. Ele realizou uma viagem ao espaço em 12 de abril de 1961, durante a missão Vostok 1. A bordo do veículo espacial Vostok 3KA-9, Gagarin completou uma órbita ao redor da Terra, permanecendo no espaço por cerca de 108 minutos.
